In [1]:
# Generate training dataset and test dataset.
from constant import *
import os
import numpy as np
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import copy
from matplotlib.pyplot import imshow
%matplotlib inline

#### Parameters. ####
is_crop = False
#####################

# Load labels.
imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
label_to_wnid = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')
wnid_to_label = {}
for label in range(len(label_to_wnid)):
    wnid = label_to_wnid[label].split(' ')[0]
    wnid_to_label[wnid] = label

dataset = [(0.0, 0), (1.0/4, 4), (1.0/3, 3), (1.0/2, 3), (2.0/3, 3), (4.0/5, 3), (1.0, 1)]
training_dataset = dataset
test_dataset = dataset

if is_crop:
    crop_str = 'crop'
else:
    crop_str = 'nocrop'

train_folders = []
test_folders = []
train_files = []
test_files = []

for (occlu_size, occlu_num) in training_dataset:
    percent = str(int(100 * occlu_size))
    f = open('{}dataset/train_{}_{}.txt'.format(imagenet_root, crop_str, percent), 'w')
    train_files.append(f)
    
    folder = '{}dataset/train_{}_{}/'.format(imagenet_root, crop_str, percent)
    if not os.path.exists(folder):
        os.makedirs(folder)
    train_folders.append(folder)
        

for (occlu_size, occlu_num) in test_dataset:
    percent = str(int(100 * occlu_size))
    f = open('{}dataset/test_{}_{}.txt'.format(imagenet_root, crop_str, percent), 'w')
    test_files.append(f)

    folder = '{}dataset/test_{}_{}/'.format(imagenet_root, crop_str, percent)
    if not os.path.exists(folder):
        os.makedirs(folder)
    test_folders.append(folder)
    
# occluder size = occlu_size * occlu_size
# occluder num = occlu_num * occlu_num
# path = 'imagenet_root/dataset/train_0/name'
# {wnid_imgid}_{crop/nocrop}_{rect_i(ifcrop)}_{occlu_size}_{i}_{j}
def generate_datum(img_orig, path, f, class_id, rects, occlu_size, occlu_num):
    if occlu_size == 0:
        if is_crop:
            for rect_i, rect in enumerate(rects):
                img = copy.copy(img_orig)
                img = img.crop(rect)
                datum_path = '{}_{}_{}_0_0_0.jpeg'.format(path, crop_str, rect_i)
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            datum_path = '{}_{}_0_0_0_0.jpeg'.format(path, crop_str)
            img_orig.save(datum_path)
            f.write('{} {}\n'.format(datum_path, str(class_id)))
        return
    if is_crop:
        for rect_i, rect in enumerate(rects):
             for i in range(occlu_num):
                 for j in range(occlu_num):
                     img = copy.copy(img_orig)
                     d = ImageDraw.Draw(img)
                     if (occlu_num == 1):
                         delta = 1
                     else:
                         delta = (1 - occlu_size) / float(occlu_num - 1)
                     subrect = [0, 0, 0, 0]
                     subrect[0] = rect[0] + i * (rect[2] - rect[0]) * delta
                     subrect[1] = rect[1] + j * (rect[3] - rect[1]) * delta
                     subrect[2] = subrect[0] + (rect[2] - rect[0]) * occlu_size
                     subrect[3] = subrect[1] + (rect[3] - rect[1]) * occlu_size
                     d.rectangle(subrect, fill="black", outline=None)
                     img = img.crop(rect)
                     datum_path = '{}_{}_{}_{}_{}_{}.jpeg'.format(path, crop_str, rect_i, str(int(100 * occlu_size)), str(i), str(j))
                     img.save(datum_path)
                     f.write('{} {}\n'.format(datum_path, str(class_id)))
        return
    for i in range(occlu_num):
        for j in range(occlu_num):
            img = copy.copy(img_orig)
            d = ImageDraw.Draw(img)
            if (occlu_num == 1):
                delta = 1
            else:
                delta = (1 - occlu_size) / float(occlu_num - 1)
            for rect in rects:
                subrect = [0, 0, 0, 0]
                subrect[0] = rect[0] + i * (rect[2] - rect[0]) * delta
                subrect[1] = rect[1] + j * (rect[3] - rect[1]) * delta
                subrect[2] = subrect[0] + (rect[2] - rect[0]) * occlu_size
                subrect[3] = subrect[1] + (rect[3] - rect[1]) * occlu_size
                d.rectangle(subrect, fill="black", outline=None)
            datum_path = '{}_{}_{}_{}_{}_{}.jpeg'.format(path, crop_str, 0, str(int(100 * occlu_size)), str(i), str(j))
            img.save(datum_path)
            f.write('{} {}\n'.format(datum_path, str(class_id)))
            
synset_names = os.listdir(imagenet_root + 'image/')
             
for synset_name in synset_names:
    image_names = os.listdir(imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images')
    annotation_names = os.listdir(imagenet_root + 'Annotation/' + synset_name + '/')
    n1 = [os.path.splitext(n)[0] for n in image_names]
    n2 = [os.path.splitext(n)[0] for n in annotation_names]
    intersection_names = list(set(n1) & set(n2))
    # train : test = 300 : 100
    for i in range(400):
        print 'Processing: ', intersection_names[i]
        # Read bounding box.
        bbx_file = open(imagenet_root + 'Annotation/' + synset_name + '/' + intersection_names[i] + '.xml')
        xmltree = ET.parse(bbx_file)
        objects = xmltree.findall('object')
        rects = []
        for obj in objects:
            bbx = obj.find('bndbox')
            rects.append([int(it.text) for it in bbx])
            
        img_orig = Image.open(imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images/' + intersection_names[i] + '.JPEG')
        if i < 300: # Training dataset. 
            for index, (occlu_size, occlu_num) in enumerate(training_dataset):
                generate_datum(img_orig, '{}{}'.format(train_folders[index], intersection_names[i]), \
                               train_files[index], original_to_new_class_id[wnid_to_label[synset_name]], rects, occlu_size, occlu_num)
        else: # Testing dataset.
            for index, (occlu_size, occlu_num) in enumerate(test_dataset):
                generate_datum(img_orig, '{}{}'.format(test_folders[index], intersection_names[i]), \
                               test_files[index], original_to_new_class_id[wnid_to_label[synset_name]], rects, occlu_size, occlu_num)
            
for f in train_files:
    f.close()
for f in test_files:
    f.close()
            


Processing:  n01855672_7535
Processing:  n01855672_10519
Processing:  n01855672_9651
Processing:  n01855672_9650
Processing:  n01855672_1945
Processing:  n01855672_10696
Processing:  n01855672_1109
Processing:  n01855672_9894
Processing:  n01855672_5916
Processing:  n01855672_3292
Processing:  n01855672_15408
Processing:  n01855672_9486
Processing:  n01855672_13029
Processing:  n01855672_12823
Processing:  n01855672_193
Processing:  n01855672_190
Processing:  n01855672_5300
Processing:  n01855672_4783
Processing:  n01855672_5628
Processing:  n01855672_13410
Processing:  n01855672_5537
Processing:  n01855672_13419
Processing:  n01855672_13418
Processing:  n01855672_1723
Processing:  n01855672_10346
Processing:  n01855672_6133
Processing:  n01855672_1937
Processing:  n01855672_1728
Processing:  n01855672_1931
Processing:  n01855672_3299
Processing:  n01855672_3586
Processing:  n01855672_3581
Processing:  n01855672_6037
Processing:  n01855672_6038
Processing:  n01855672_2626
Processing:  

In [11]:
# Data cleaning.
'''
Some constants:

synset_names = ['n01855672', 'n02009912', 'n02051845', 'n02398521', 'n03017168',\
'n03028079', 'n03109150', 'n03160309', 'n03179701', 'n03759954', 'n03770679', \
'n03983396', 'n04086273', 'n04372370', 'n04392985', 'n04584207', 'n07720875', \
'n02504458', 'n02276258', 'n01944390']

new_to_original_class_id = [99, 113, 132, 144, 321, 344, 386, 494, 497, 512, 525, 526, \
650, 656, 737, 763, 844, 848, 903, 945]

original_to_new_class_id = {512: 9, 321: 4, 386: 6, 99: 0, 132: 2, 848: 17, 497: 8,\
650: 12, 903: 18, 844: 16, 525: 10, 494: 7, 144: 3, 113: 1, 763: 15, 526: 11, 344: 5,\
656: 13, 737: 14, 945: 19}

'''

import os
import random

imagenet_root = '/home/haow3/occlusion-project/data/imagenet/'
caffe_root = '/home/haow3/software/caffe-rc3/'

original_to_new_class_id = {512: 9, 321: 4, 386: 6, 99: 0, 132: 2, 848: 17, 497: 8,\
650: 12, 903: 18, 844: 16, 525: 10, 494: 7, 144: 3, 113: 1, 763: 15, 526: 11, 344: 5,\
656: 13, 737: 14, 945: 19}

'''
for filename in os.listdir(imagenet_root + 'dataset/list_original_id/'):
    if filename.find('.txt') == -1:
        continue
    print 'Processing: ', filename
    
    fin = open(imagenet_root + 'dataset/list_original_id/' + filename, 'r')
    fout = open(imagenet_root + 'dataset/' + filename, 'w')
    
    lines = fin.readlines()
    for line in lines:
        filepath, class_id = line.split(' ')
        fout.write('{} {}\n'.format(filepath, original_to_new_class_id[int(class_id)]))
        
    fin.close()
    fout.close()
'''
f = open(imagenet_root + 'dataset/train_crop_all_unsampled.txt', 'r')
lines = f.readlines()
f.close()
print len(lines)
sampled = random.sample(lines, 62316)
print len(sampled)

f = open(imagenet_root + 'dataset/train_crop_all.txt', 'w')
f.writelines(sampled)
f.close()

325428
62316
[8, 9, 7, 4, 3, 0]
